## Importing Modules Required (We will also import whenever required!!!)

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Using Transfer Learning Technique for Data Preprocessing

In [5]:
# We are using InceptionResNetV2 Convolutional Neural Network for extracting features.
# I'm using InceptionResNetV2 because it performs better than rest of the models if you check in Keras Official Docs
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# We are preprocessing the input to be given for better results.
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [6]:
# We use ImageDataGenerator for reading images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Data Augmentation for Better Performance

In [7]:
# We use the same ImageDataGenerator for augmenting images which means zooming, stretching, sliding the images.
train_image_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                     rotation_range=360,
                                     width_shift_range=0.4,
                                    height_shift_range=0.4,
                                    zoom_range=0.5,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    shear_range=0.3)

In [11]:
# Reading input images with the help of flow_from_directory function. 
# We commonly specify the tensor size of image(literally image size) i.e target size as (299,299) which will be 
# good for training the model.
train_images = train_image_gen.flow_from_directory('./train',batch_size=32,target_size=(299,299))

Found 4750 images belonging to 12 classes.


In [12]:
# Observing the classes of training data.
train_images.class_indices

{'Black-grass': 0,
 'Charlock': 1,
 'Cleavers': 2,
 'Common Chickweed': 3,
 'Common wheat': 4,
 'Fat Hen': 5,
 'Loose Silky-bent': 6,
 'Maize': 7,
 'Scentless Mayweed': 8,
 'Shepherds Purse': 9,
 'Small-flowered Cranesbill': 10,
 'Sugar beet': 11}

## Data is ready!!! Let's create a model for training the image data.

In [13]:
# First we are giving the input tensor to CNNs.
from tensorflow.keras.layers import Input
# Here 3 in the input shape denotes there are three channels for every image which are Red, Green and Blue.
primary_input = Input(shape=(299,299,3))

#### Initiating the base model and giving input to it. We don't want top dense layers of InceptionResNetV2 as it deals with another task, so we set 'include_top' parameter to False. We will give top layers separately according to our task specified - This is known as TRANSFER lEARNING.

In [19]:
# Here I'm using the Funtional API of Keras for dealing with layers initialization.
base_model = InceptionResNetV2(include_top=False,input_shape=(299,299,3))(primary_input)

In [20]:
base_model.shape

TensorShape([None, 8, 8, 1536])

In [21]:
# Flattening the base_model tensor so as we can give the flattened array to Dense Layers.
from tensorflow.keras.layers import Flatten
vector_representation = Flatten()(base_model)

In [22]:
# See how nicely it is flattened.
vector_representation

<KerasTensor: shape=(None, 98304) dtype=float32 (created by layer 'flatten_1')>

## Initiating the dense layers which I mentioned before as top layers.

In [23]:
from tensorflow.keras.layers import Dense
# Created a dense layer with 128 neurons and given the vector as input for dense layer.
dense_layer = Dense(128, activation='relu')(vector_representation)
# Created another layer with 12 neurons as we have 12 classes to classify an image which results as output.
output = Dense(12, activation='softmax')(dense_layer)

In [24]:
# Check the output whether it is having 12 columns
print(output)  # Yeah we have!!!

KerasTensor(type_spec=TensorSpec(shape=(None, 12), dtype=tf.float32, name=None), name='dense_1/Softmax:0', description="created by layer 'dense_1'")


## All layers in the neural network has been set. Now Let's create our final model for training and predicting!!!

In [25]:
from tensorflow.keras.models import Model
model = Model(primary_input, output)

In [26]:
# Optimizing the learning rate with Adam Optimizer which is considerably better suited.
from tensorflow.keras.optimizers import Adam
# You can tune the learning rate hyperparameter as you wish.
optimizer = Adam(0.001)  
# Let's compile our model. Here we are specifying 'loss' as categorical_crossentropy because our task is multi-class 
# image classification.
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## Model set!! What are we waiting for???!!! Let's train the model by fitting the training data.

In [28]:
model.fit(train_images,batch_size=32,epochs=20,verbose=1)

Epoch 1/20
149/149 [==============================] - 161s 1s/step - loss: 3.0053 - accuracy: 0.1145
Epoch 2/20
149/149 [==============================] - 153s 1s/step - loss: 2.3686 - accuracy: 0.1531
Epoch 3/20
149/149 [==============================] - 157s 1s/step - loss: 1.7998 - accuracy: 0.3813
Epoch 4/20
149/149 [==============================] - 162s 1s/step - loss: 1.3439 - accuracy: 0.5514
Epoch 5/20
149/149 [==============================] - 162s 1s/step - loss: 1.0709 - accuracy: 0.6518
Epoch 6/20
149/149 [==============================] - 162s 1s/step - loss: 0.7237 - accuracy: 0.7558
Epoch 7/20
149/149 [==============================] - 157s 1s/step - loss: 0.6212 - accuracy: 0.7863
Epoch 8/20
149/149 [==============================] - 152s 1s/step - loss: 0.5189 - accuracy: 0.8227
Epoch 9/20
149/149 [==============================] - 155s 1s/step - loss: 0.5055 - accuracy: 0.8246
Epoch 10/20
149/149 [==============================] - 162s 1s/step - loss: 0.4465 - accura

In [29]:
# Reading the test images to be classified with the help of flow_from_directory method from ImageDataGenerator class
test_image_gen = ImageDataGenerator()

## Never forget to maintain all the test images under a single sub-directory in a directory because Keras treats a sub-folder as a separate class.

In [30]:
# We are mentioning the same target size as before.
# Coming to 'shuffle' parameter, I don't want the order of test images to be changed so I set it to False.
# Mentioning 'batch_size' parameter as 1 because I want Keras to go through one test image for every moment. You can
# mention 32 if you want Keras to consider 32 images at a time.
test_images = test_image_gen.flow_from_directory('./test_images',target_size=(299,299),shuffle=False,batch_size=1)

Found 794 images belonging to 1 classes.


## Training done. Predict the test images with the help of our model.

In [31]:
predictions = model.predict(test_images)

794/794 [==============================] - 29s 31ms/step


In [32]:
# There should be 794 rows representing 794 images and 12 columns representing 12 classes of plant species.
print(predictions.shape)  # Woohoo!! Correct.

(794, 12)


In [33]:
# Predictions show the probability of every image(row) belonging to that particular class(column) among the 12.
predictions

array([[2.8512020e-05, 3.8338217e-09, 2.4022557e-07, ..., 2.0378469e-05,
        3.0488253e-04, 3.0773869e-08],
       [5.8171154e-06, 9.3529451e-08, 3.1227860e-06, ..., 1.4535377e-03,
        1.8914023e-05, 2.0412527e-07],
       [5.7633162e-07, 9.6632086e-12, 2.6107316e-09, ..., 2.1695488e-04,
        1.4394147e-03, 5.8251709e-11],
       ...,
       [2.3282331e-05, 1.4414688e-08, 3.7846653e-06, ..., 2.1360086e-04,
        1.3802096e-03, 6.1799554e-08],
       [4.9882534e-09, 4.4837519e-11, 1.1932534e-08, ..., 8.1049868e-05,
        1.0026866e-02, 8.8916358e-10],
       [3.8957673e-06, 7.5010540e-09, 3.2410921e-05, ..., 8.6403816e-05,
        1.5669788e-05, 2.4188955e-08]], dtype=float32)

In [34]:
# So we have to identify index of the highest probability in every row implies that particular image is having 
# higher chances to belong to that plant species(column). We have argmax to do that.
predictions[0].argmax(axis=0)

8

In [35]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]

class_list = []

# Identifying highest probability for all images and assigning it to the class it belongs.
for i in range(0, 794):
    y_class = predictions[i, :].argmax(axis=0)
    class_list += [species_list[y_class]]

In [36]:
# Creating Pandas Dataframe and creating required features.
submission = pd.DataFrame()
submission['file'] = test_images.filenames
submission['file'] = submission['file'].str.replace('test/', '')
submission['species'] = class_list

In [37]:
# Converting the dataframe into csv_file for storing it permanently.
submission.to_csv('submission.csv', index=False)

print('Submission file generated. All done.')

Submission file generated. All done.


## You can evaluate the predictions with the true values if you have true values already!! Predict function will do the task  - Finally Thank You